# Data fetching & processing


### Code is adapted from https://github.com/AI4Finance-Foundation/FinRL

# Part 1. Import Packages

In [1]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

### Using yfinance

In [2]:
aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

[*********************100%***********************]  1 of 1 completed


In [3]:
aapl_df_yf.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.449394,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.735321,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,73.314880,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.970078,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,74.143898,132079200


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [4]:
aapl_df_finrl = YahooDownloader(start_date = '2020-01-01',
                                end_date = '2020-01-31',
                                ticker_list = ['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (20, 8)


In [5]:
aapl_df_finrl.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-02,74.059998,75.150002,73.797501,73.449394,135480400,aapl,3
1,2020-01-03,74.287498,75.144997,74.125000,72.735329,146322800,aapl,4
2,2020-01-06,73.447502,74.989998,73.187500,73.314903,118387200,aapl,0
3,2020-01-07,74.959999,75.224998,74.370003,72.970078,108872000,aapl,1
4,2020-01-08,74.290001,76.110001,74.290001,74.143906,132079200,aapl,2


## Data for the chosen tickers

In [6]:
# config_tickers.DOW_30_TICKER

In [7]:
TRAIN_START_DATE = '2012-01-01'
TRAIN_END_DATE = '2022-07-01'
TRADE_START_DATE = '2022-07-01'
TRADE_END_DATE = '2023-04-12'

In [8]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2012-01-03,14.621429,14.732143,14.607143,12.500191,302220800,AAPL,1
1,2012-01-03,64.949997,65.190002,63.450001,48.132042,10216800,AMGN,1
2,2012-01-03,48.299999,48.959999,48.139999,41.038811,6955400,AXP,1
3,2012-01-03,74.699997,75.000000,74.120003,60.731102,6859300,BA,1
4,2012-01-03,92.769997,95.110001,92.769997,69.014992,8177000,CAT,1


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [10]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [23]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2835, 8)
Successfully added vix
Successfully added turbulence index


In [24]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [25]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2012-01-03,AAPL,14.621429,14.732143,14.607143,12.500195,302220800.0,1.0,0.0,12.628789,12.438779,100.0,66.666667,100.0,12.500195,12.500195,22.969999,0.0
1,2012-01-03,AMGN,64.949997,65.190002,63.450001,48.132034,10216800.0,1.0,0.0,12.628789,12.438779,100.0,66.666667,100.0,48.132034,48.132034,22.969999,0.0
2,2012-01-03,AXP,48.299999,48.959999,48.139999,41.038830,6955400.0,1.0,0.0,12.628789,12.438779,100.0,66.666667,100.0,41.038830,41.038830,22.969999,0.0
3,2012-01-03,BA,74.699997,75.000000,74.120003,60.731083,6859300.0,1.0,0.0,12.628789,12.438779,100.0,66.666667,100.0,60.731083,60.731083,22.969999,0.0
4,2012-01-03,CAT,92.769997,95.110001,92.769997,69.014999,8177000.0,1.0,0.0,12.628789,12.438779,100.0,66.666667,100.0,69.014999,69.014999,22.969999,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [26]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

76589
5626


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [27]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')

## Creating additional datasets

In [2]:
TRAIN_START_DATE = '2018-01-01'
TRAIN_END_DATE = '2022-07-01'
TRADE_START_DATE = '2022-07-01'
TRADE_END_DATE = '2023-04-12'

In [3]:
config_tickers.NAS_100_TICKER[-25:]

['CSX',
 'NTES',
 'MCHP',
 'CTAS',
 'KLAC',
 'HAS',
 'JBHT',
 'IDXX',
 'WYNN',
 'MELI',
 'ALGN',
 'CDNS',
 'WDAY',
 'SNPS',
 'ASML',
 'TTWO',
 'PEP',
 'NXPI',
 'XEL',
 'AMD',
 'NTAP',
 'VRSN',
 'LULU',
 'WLTW',
 'UAL']

In [32]:
new_list = ['CSX',
 'NTES',
 'MCHP',
 'CTAS',
 'KLAC',
 'HAS',
 'JBHT',
 'IDXX',
 'WYNN',
 'MELI',
 'ALGN',
 'CDNS',
 'WDAY',
 'SNPS',
 'ASML',
 'TTWO',
 'PEP',
 'NXPI',
 'XEL',
 'AMD',
 'NTAP',
 'VRSN',
 'LULU',
 'XPEV',
 'UAL',
 'BABA',
 'NIO',
 'JD',
 'GME',
 'EDU',
 'TSM']

In [33]:
len(new_list)

31

In [34]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = new_list).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [35]:
len(df_raw['tic'].unique())

31

In [36]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)
#some stocks get filtered out

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1326, 8)
Successfully added vix
Successfully added turbulence index


In [37]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [43]:
list_ticker

['ALGN',
 'AMD',
 'ASML',
 'BABA',
 'CDNS',
 'CSX',
 'CTAS',
 'EDU',
 'GME',
 'HAS',
 'IDXX',
 'JBHT',
 'JD',
 'KLAC',
 'LULU',
 'MCHP',
 'MELI',
 'NTAP',
 'NTES',
 'NXPI',
 'PEP',
 'SNPS',
 'TSM',
 'TTWO',
 'UAL',
 'VRSN',
 'WDAY',
 'WYNN',
 'XEL']

In [38]:
len(list_ticker)

29

In [39]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2018-01-02,ALGN,222.610001,226.190002,220.360001,224.619995,979700.0,1.0,0.0,237.215524,218.604468,100.0,66.666667,100.0,224.619995,224.619995,9.77,0.0
1,2018-01-02,AMD,10.420000,11.020000,10.340000,10.980000,44146300.0,1.0,0.0,237.215524,218.604468,100.0,66.666667,100.0,10.980000,10.980000,9.77,0.0
2,2018-01-02,ASML,174.139999,177.839996,173.979996,168.355408,818600.0,1.0,0.0,237.215524,218.604468,100.0,66.666667,100.0,168.355408,168.355408,9.77,0.0
3,2018-01-02,BABA,176.399002,184.100006,175.699997,183.649994,29916900.0,1.0,0.0,237.215524,218.604468,100.0,66.666667,100.0,183.649994,183.649994,9.77,0.0
4,2018-01-02,CDNS,42.000000,42.209999,41.709999,42.049999,1935300.0,1.0,0.0,237.215524,218.604468,100.0,66.666667,100.0,42.049999,42.049999,9.77,0.0


In [40]:
train_2 = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade_2 = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train_2))
print(len(trade_2))

32828
5626


In [41]:
train_2.to_csv('train_data_2.csv')
trade_2.to_csv('trade_data_2.csv')

In [42]:
len(trade_2['tic'].unique())

29